In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

In [2]:
glockannFile='/home/m/glock/annotations_glock/trainval.json'
cocoannFile='/home/m/glock/annotations_coco/train2017.json'

In [3]:
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')
class CustomCOCO(COCO):
    def __init__(self, annotation_file=None):
        super().__init__(annotation_file)
        
    def getallImgIds(self, imgIds=[], catIds=[]):
        '''
        Get img ids that satisfy given filter conditions.
        :param imgIds (int array) : get imgs for given ids
        :param catIds (int array) : get imgs with all given cats
        :return: ids (int array)  : integer array of img ids
        '''
        imgIds = imgIds if _isArrayLike(imgIds) else [imgIds]
        catIds = catIds if _isArrayLike(catIds) else [catIds]

        if len(imgIds) == len(catIds) == 0:
            ids = self.imgs.keys()
        else:
            ids = imgIds
            for i, catId in enumerate(catIds):
                ids += self.catToImgs[catId]
        return list(set(ids))
    
# initialize COCO api for instance annotations
glock=CustomCOCO(glockannFile)
coco=CustomCOCO(cocoannFile)

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=12.27s)
creating index...
index created!


In [4]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

COCO supercategories: 
appliance furniture animal sports food kitchen indoor accessory vehicle electronic outdoor person


In [5]:
# get all images containing given categories, select one at random
print(coco.getCatIds(catNms=['knife']))

coco_catIds = coco.getCatIds(catNms=['person', 'knife']);
print(coco_catIds)
coco_cats = coco.loadCats(coco_catIds);
print(coco_cats)

[49]
[1, 49]
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'kitchen', 'id': 49, 'name': 'knife'}]


In [6]:
glock_catIds = glock.getCatIds();
glock_cats = glock.loadCats(glock_catIds);

In [7]:
coco_annIds = coco.getAnnIds(catIds=coco_catIds)
print(len(coco_annIds))
coco_anns = coco.loadAnns(coco_annIds)

270235


In [8]:
glock_annIds = glock.getAnnIds(catIds=glock_catIds)
glock_anns = glock.loadAnns(glock_annIds)

In [24]:
coco_imgIds = coco.getallImgIds(catIds=coco_catIds)
print(len(coco_imgIds))
coco_imgs = coco.loadImgs(coco_imgIds)
print(len(coco_imgs))

66081
66081
[{'license': 2, 'file_name': '000000262145.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262145.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-20 02:07:55', 'flickr_url': 'http://farm8.staticflickr.com/7187/6967031859_5f08387bde_z.jpg', 'id': 262145}, {'license': 1, 'file_name': '000000262146.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262146.jpg', 'height': 640, 'width': 480, 'date_captured': '2013-11-19 23:07:16', 'flickr_url': 'http://farm6.staticflickr.com/5090/5341741494_1f653cdb80_z.jpg', 'id': 262146}, {'license': 3, 'file_name': '000000524291.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000524291.jpg', 'height': 426, 'width': 640, 'date_captured': '2013-11-18 09:59:07', 'flickr_url': 'http://farm2.staticflickr.com/1045/934293170_d1b2cc58ff_z.jpg', 'id': 524291}, {'license': 1, 'file_name': '000000262148.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262148.jpg', 'height': 512, '

In [10]:
glock_imgIds = glock.getImgIds()
glock_imgs = glock.loadImgs(glock_imgIds)

In [11]:
out_json=dict()

In [12]:
out_json['categories'] = coco_cats + glock_cats

In [13]:
out_json['images'] = coco_imgs+glock_imgs

In [14]:
out_json['annotations'] = coco_anns+glock_anns

In [15]:
print(out_json['categories'])

[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'kitchen', 'id': 49, 'name': 'knife'}, {'supercategory': 'gun', 'name': 'glock', 'id': 2}]


In [16]:
import json
with open('/home/m/glock/glock_person_anns/train.json', 'w') as fp:
    json.dump(out_json, fp)

In [25]:
coco_image_ids = coco.getallImgIds(catIds=coco_catIds)
coco_image_data = coco.loadImgs(coco_image_ids)

In [26]:
print(coco_image_data[0])

{'license': 2, 'file_name': '000000262145.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000262145.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-20 02:07:55', 'flickr_url': 'http://farm8.staticflickr.com/7187/6967031859_5f08387bde_z.jpg', 'id': 262145}


In [27]:
import shutil
import tqdm
import os

coco_imgs_dir='/home/m/glock/images_coco/train2017'
output_imgs_dir='/home/m/glock/glock_person_imgs'
if os.path.exists(output_imgs_dir):
    shutil.rmtree(output_imgs_dir)

In [28]:
def copy_files(image_data, input_imgs_dir, output_imgs_dir):
    for data in tqdm.tqdm(image_data):
        image_name=data['file_name']
        output_path=os.path.join(output_imgs_dir, image_name)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        shutil.copy(os.path.join(input_imgs_dir, image_name), output_path)

In [29]:
copy_files(coco_image_data, coco_imgs_dir, output_imgs_dir)

100%|██████████| 66081/66081 [00:22<00:00, 2983.41it/s]


In [30]:
glock_dir='/home/m/train_glock_unity/'

In [31]:
copy_files(glock_imgs, glock_dir, output_imgs_dir)

100%|██████████| 3889/3889 [00:01<00:00, 3183.06it/s]
